In [1]:
from kamgon import *

In [2]:
class SudokuProblem(CSPProblem):
    """
    ปัญหา Sudoku ที่ต้องการกรอกตัวเลข 1-9 ในตาราง 9x9 
    โดยแต่ละแถว แต่ละคอลัมน์ และแต่ละกล่อง 3x3 ต้องมีตัวเลข 1-9 ครบทุกตัว
    """
    
    def __init__(self, puzzle: List[List[int]]):
        """
        เริ่มต้นปัญหา Sudoku
        
        Args:
            puzzle: ตาราง 9x9 โดยช่องว่างแสดงด้วย 0
        """
        self.puzzle = puzzle
        variables = []
        domains = {}
        
        for row in range(9):
            for col in range(9):
                if puzzle[row][col] == 0:  # ช่องว่างที่ต้องกรอก
                    var_name = f"cell_{row}_{col}"
                    variables.append(var_name)
                    domains[var_name] = list(range(1, 10))
        
        super().__init__(variables, domains)
    
    def is_consistent(self, variable: str, value: Any, assignment: Dict[str, Any]) -> bool:
        """
        ตรวจสอบว่าการกรอกตัวเลขในช่องนี้ขัดแย้งกับกฎ Sudoku หรือไม่
        
        Args:
            variable: ชื่อตัวแปรที่แสดงถึงช่องในตาราง
            value: ตัวเลขที่จะกรอก
            assignment: การกรอกตัวเลขที่มีอยู่แล้ว
            
        Returns:
            True ถ้าสามารถกรอกตัวเลขนี้ได้โดยไม่ขัดแย้ง
        """
        row, col = self._parse_variable(variable)
        
        # สร้างตารางปัจจุบันรวมกับการกำหนดค่าใหม่
        current_board = [row[:] for row in self.puzzle]
        for var, val in assignment.items():
            if var != variable:
                var_row, var_col = self._parse_variable(var)
                current_board[var_row][var_col] = val
        
        # ตรวจสอบแถว
        for c in range(9):
            if c != col and current_board[row][c] == value:
                return False
        
        # ตรวจสอบคอลัมน์
        for r in range(9):
            if r != row and current_board[r][col] == value:
                return False
        
        # ตรวจสอบกล่อง 3x3
        box_row, box_col = 3 * (row // 3), 3 * (col // 3)
        for r in range(box_row, box_row + 3):
            for c in range(box_col, box_col + 3):
                if (r != row or c != col) and current_board[r][c] == value:
                    return False
        
        return True
    
    def _parse_variable(self, variable: str) -> tuple:
        """แยกชื่อตัวแปรเพื่อหา row และ column"""
        parts = variable.split('_')
        return int(parts[1]), int(parts[2])

In [7]:
# การใช้งาน Sudoku Problem
if __name__ == "__main__":
    print("\n=== Sudoku Problem ===")
    
    # ตัวอย่างปัญหา Sudoku (0 แสดงช่องว่าง)
    sudoku_puzzle = [
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]
    ]
    
    sudoku = SudokuProblem(sudoku_puzzle)
    
    print("ปัญหา Sudoku เริ่มต้น:")
    for row in sudoku_puzzle:
        print(' '.join(str(cell) if cell != 0 else '.' for cell in row))
    
    # แก้ปัญหาด้วย backtracking
    solution = backtracking_search(sudoku, verbose=True)
    
    if solution:
        print("\nคำตอบ Sudoku:")
        solved_board = [row[:] for row in sudoku_puzzle]
        for var, val in solution.items():
            row, col = sudoku._parse_variable(var)
            solved_board[row][col] = val
        
        for row in solved_board:
            print(' '.join(str(cell) for cell in row))
    else:
        print("ไม่พบคำตอบ")


=== Sudoku Problem ===
ปัญหา Sudoku เริ่มต้น:
5 3 . . 7 . . . .
6 . . 1 9 5 . . .
. 9 8 . . . . 6 .
8 . . . 6 . . . 3
4 . . 8 . 3 . . 1
7 . . . 2 . . . 6
. 6 . . . . 2 8 .
. . . 4 1 9 . . 5
. . . . 8 . . 7 9
กำลังสำรวจ node: 0/51 ตัวแปรได้รับการกำหนดค่า
กำลังลอง cell_4_4 = 5
  กำลังสำรวจ node: 1/51 ตัวแปรได้รับการกำหนดค่า
  กำลังลอง cell_4_1 = 2
    กำลังสำรวจ node: 2/51 ตัวแปรได้รับการกำหนดค่า
    กำลังลอง cell_4_7 = 9
      กำลังสำรวจ node: 3/51 ตัวแปรได้รับการกำหนดค่า
      กำลังลอง cell_4_2 = 6
        กำลังสำรวจ node: 4/51 ตัวแปรได้รับการกำหนดค่า
        กำลังลอง cell_4_6 = 7
          กำลังสำรวจ node: 5/51 ตัวแปรได้รับการกำหนดค่า
          กำลังลอง cell_5_3 = 9
            กำลังสำรวจ node: 6/51 ตัวแปรได้รับการกำหนดค่า
            กำลังลอง cell_3_3 = 7
              กำลังสำรวจ node: 7/51 ตัวแปรได้รับการกำหนดค่า
              กำลังลอง cell_6_4 = 3
                กำลังสำรวจ node: 8/51 ตัวแปรได้รับการกำหนดค่า
                กำลังลอง cell_2_4 = 4
                  กำลังสำรวจ node: 

In [6]:
# การใช้งาน Sudoku Problem
if __name__ == "__main__":
    print("\n=== Sudoku Problem ===")
    
    # ตัวอย่างปัญหา Sudoku (0 แสดงช่องว่าง)
    sudoku_puzzle = [
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]
    ]
    
    sudoku = SudokuProblem(sudoku_puzzle)
    
    print("ปัญหา Sudoku เริ่มต้น:")
    for row in sudoku_puzzle:
        print(' '.join(str(cell) if cell != 0 else '.' for cell in row))

  
    initial_state = sudoku.initial_state
    goal_state = CSPState()    
    result_node = best_first_search(
            sudoku, 
            initial_state, 
            goal_state,
            f=lambda node, goal: sudoku.heuristic(node.state, goal),
            verbose=True
        )
    
    if solution:
        print("\nคำตอบ Sudoku:")
        solved_board = [row[:] for row in sudoku_puzzle]
        for var, val in solution.items():
            row, col = sudoku._parse_variable(var)
            solved_board[row][col] = val
        
        for row in solved_board:
            print(' '.join(str(cell) for cell in row))
    else:
        print("ไม่พบคำตอบ")


=== Sudoku Problem ===
ปัญหา Sudoku เริ่มต้น:
5 3 . . 7 . . . .
6 . . 1 9 5 . . .
. 9 8 . . . . 6 .
8 . . . 6 . . . 3
4 . . 8 . 3 . . 1
7 . . . 2 . . . 6
. 6 . . . . 2 8 .
. . . 4 1 9 . . 5
. . . . 8 . . 7 9
1. โหนดปัจจุบัน: <Node CSPState({})>, f(n)=51
  -> เพิ่มไปยัง waitlist: CSPState({'cell_4_4': 5}) ด้วย f(n)=50
 :: waitlist ปัจจุบัน: <kamgon.PriorityQueue object at 0x000001F4B0A075E0>
2. โหนดปัจจุบัน: <Node CSPState({'cell_4_4': 5})>, f(n)=50
  -> เพิ่มไปยัง waitlist: CSPState({'cell_4_4': 5, 'cell_4_1': 2}) ด้วย f(n)=49
 :: waitlist ปัจจุบัน: <kamgon.PriorityQueue object at 0x000001F4B0A075E0>
3. โหนดปัจจุบัน: <Node CSPState({'cell_4_4': 5, 'cell_4_1': 2})>, f(n)=49
  -> เพิ่มไปยัง waitlist: CSPState({'cell_4_4': 5, 'cell_4_1': 2, 'cell_4_7': 9}) ด้วย f(n)=48
 :: waitlist ปัจจุบัน: <kamgon.PriorityQueue object at 0x000001F4B0A075E0>
4. โหนดปัจจุบัน: <Node CSPState({'cell_4_4': 5, 'cell_4_1': 2, 'cell_4_7': 9})>, f(n)=48
  -> เพิ่มไปยัง waitlist: CSPState({'cell_4_4': 5, 'cell_4